In [1]:
include("../src/KagomeTransport.jl");
using .KagomeTransport.Kagome_Hamiltonian;
using LinearAlgebra;
using StaticArrays;


In [5]:
"""
Calculate the xy comp. of the complex quantum metric (both the real quantum geometry and berry curvature of the bands at a single point in the BZ)
"""
@inline function Complex_Metric_xy(;
    E::Vector{Float64}, 
    U::Matrix{ComplexF64}, 
    Vxb::Matrix{ComplexF64}, 
    Vyb::Matrix{ComplexF64},
    Num_Bands::Int64)
    
    ωmn = E' .- E
    ωmn[ωmn .== 0] .= 1e-15
    
    setindex!.(Ref(Vxb), 0.0 + im * 0.0, 1:Num_Bands, 1:Num_Bands)
    setindex!.(Ref(Vyb), 0.0 + im * 0.0, 1:Num_Bands, 1:Num_Bands)

    Xmn_x = -im * (Vxb ./ ωmn); Xmn_y = -im * (Vyb ./ ωmn);
    Xmn_x * Xmn_y'
end



Complex_Metric_xy

In [3]:
# Preliminary calculation of the Berry curvature 
let npts_perdim = 300, lim = 1.0
    
    # construction of the grid 
    rng = LinRange(-lim * pi, lim * pi, npts_perdim)
    grid = Matrix{Vector{Float64}}(undef, npts_perdim, npts_perdim)
    for (n, kx) ∈ enumerate(rng)
        for (m, ky) ∈ enumerate(rng)
            grid[n, m] = [kx, ky]
        end
    end    
    
end


In [4]:
p = Params()
(E, U) = eigensystem3c([0.0, 0.0], p)

Eigen{ComplexF64, Float64, Matrix{ComplexF64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -1.9999999999999998
 -1.9999999999999964
  3.9999999999999996
vectors:
3×3 Matrix{ComplexF64}:
  0.707107+0.0im  -0.408248+0.0im  -0.57735+0.0im
 -0.707107+0.0im  -0.408248+0.0im  -0.57735+0.0im
       0.0+0.0im   0.816497+0.0im  -0.57735-0.0im